In [253]:
import pandas as pd
import re

# KR operations

In [254]:
filexlsx = pd.read_excel("train_data/KC_TR_KR/KR on 01.01.2017 unprotected.xlsx", skiprows=5) # Skip header

columns_to_drop = [0, 2, 6, 7, 10]
filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

new_column_names = ["name", "estimated_cost", "used_year_start", "approved_year_start", "used_year_end", "expected_year_end"]
filexlsx.columns = new_column_names

filexlsx

,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end
0,NaN,NaN,NaN,Всего:\n,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3.00,4.0,5,8.00,9.00
3,Раздел 1. Объекты основной деятельности,NaN,NaN,NaN,NaN,NaN
4,Дальневосточное ГУ Банка России г.Владивосток,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
82,в том числе: проектирование,NaN,NaN,NaN,NaN,NaN
83,капремонт,9602.91,NaN,11863.74,9602.91,9602.91
84,Итого ОтделениеХабаровск,9602.91,NaN,11863.74,9602.91,9602.91
85,Итого по разделу 3,9602.91,NaN,11863.74,9602.91,9602.91


In [255]:
target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

filtered_rows

,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end
5,Капитальный ремонт подземного сооружения (Лит....,4088.43,160.95,4077.25,3927.48,3927.48
8,"Капитальный ремонт здания РКЦ Находка, располо...",19001.91,NaN,1149.83,1143.30,1143.30
13,Капитальный ремонт гаража по адресу: Амурская ...,13677.35,819.34,12858.01,12858.01,12858.01
16,Выборочный капитальный ремонт административног...,18034.12,NaN,1298.34,1298.33,1298.33
19,Выборочный капитальный ремонт административног...,448.72,NaN,478.98,448.72,448.72
24,Выборочный капитальный ремонт здания гаража на...,4439.86,NaN,199.79,199.79,199.79
29,Выборочный капитальный ремонт сетей теплоснабж...,97.37,NaN,97.37,97.37,97.37
34,Выборочный капитальный ремонт административног...,2344.49,NaN,3281.74,2344.49,2344.49
39,"Выборочный капитальный ремонт ""Сооружения комм...",184.94,NaN,184.95,184.94,184.94
42,Выборочный капитальный ремонт в части усиления...,3634.45,325.99,3621.51,3308.46,3308.46


In [256]:
def extract_locality_and_street(input_string):
    # Список подстрок, которые могут предшествовать названию населенного пункта
    prefixes = ["г.", "г ", "с.", "с ", "кп ", "к.п."]
    locality_keyword = "по адресу:"

    # Начальные значения для города и улицы
    locality = ""
    street = input_string

    locality_index = input_string.find(locality_keyword)
    input_string = input_string[locality_index:]

    # Поиск подстроки-префикса
    for prefix in prefixes:
        index = input_string.find(prefix)
        if index != -1:
            comma_index = input_string.find(',', index)
            locality = input_string[index:comma_index].strip()
            street_start = comma_index + 1
            street = input_string[street_start:].strip()
            break

    return locality, street

In [257]:
filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

filtered_rows

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/1407716207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_30125/1407716207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))


,name,estimated_cost,used_year_start,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт подземного сооружения (Лит....,4088.43,160.95,4077.25,3927.48,3927.48,г. Владивосток,"ул. Светланская, д. 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",19001.91,NaN,1149.83,1143.30,1143.30,г. Находка,"проспект Находкинский, 13"
13,Капитальный ремонт гаража по адресу: Амурская ...,13677.35,819.34,12858.01,12858.01,12858.01,г. Благовещенск,"пер. Святителя Иннокентия, 17"
16,Выборочный капитальный ремонт административног...,18034.12,NaN,1298.34,1298.33,1298.33,г. Тында,"ул. Профсоюзная, 12"
19,Выборочный капитальный ремонт административног...,448.72,NaN,478.98,448.72,448.72,г. Благовещенск,"пер. Св. Иннокентия, 17"
24,Выборочный капитальный ремонт здания гаража на...,4439.86,NaN,199.79,199.79,199.79,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
29,Выборочный капитальный ремонт сетей теплоснабж...,97.37,NaN,97.37,97.37,97.37,г. Магадан,"ул. Пролетарская, д. 8, строение 1, литера А, ..."
34,Выборочный капитальный ремонт административног...,2344.49,NaN,3281.74,2344.49,2344.49,г. Покровск,"ул. Орджоникидзе, д. 24"
39,"Выборочный капитальный ремонт ""Сооружения комм...",184.94,NaN,184.95,184.94,184.94,,"Выборочный капитальный ремонт ""Сооружения комм..."
42,Выборочный капитальный ремонт в части усиления...,3634.45,325.99,3621.51,3308.46,3308.46,г. Южно-Сахалинск,"Коммунистический проспект, 47 (литер А)"
